<a href="https://colab.research.google.com/github/sadra-barikbin/persian-information-retrieval-example/blob/main/Persian-IR-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install hazm transformers ir_measures

In [47]:
import torch
import yaml
import numpy as np
import pandas as pd
import ir_measures as IRm
from typing import List
from pathlib import Path
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Loading & Preparing Data

## Corpus

In [5]:
!wget https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip

--2021-12-28 06:06:50--  https://github.com/language-ml/2-LM-embedding-projects/raw/main/problem3/doc_collection.zip
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/doc_collection.zip [following]
--2021-12-28 06:06:51--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/doc_collection.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6083582 (5.8M) [application/zip]
Saving to: ‘doc_collection.zip’

doc_collection.zip  100%[===================>]   5.80M  6.61MB/s    in 0.9s    

2021-12-28 06:06:52 (6.61 MB/s) -

In [ ]:
!unzip doc_collection.zip

In [4]:
!cat IR_dataset/1000.txt

ببر سیبری که با نام‌های ببر آلتایی، ببر منچوری، ببر کره‌ای، ببر آمور و ببر اوسوری نیز شناخته می‌شود، یکی از زیرگونه‌های ببر است که در گذشته در بخش‌های وسیعی از شرق آسیا می‌زیست اما امروزه تنها در منطقهٔ حفاظت شده‌ای در شرق سیبری زندگی می‌کند. ببر سیبری بزرگترین زیرگونهٔ ببر و بزرگترین گربه‌سان زندهٔ جهان است. ببر منقرض شده مازندران نزدیک‌ترین زیرگونه ببر به ببر سیبری است و مطالعات ژنتیکی جدید حکایت از آن دارد که این دو را حتی می‌توان یک زیرگونه محسوب کرد.

ببر سیبری در دهه ۱۹۳۰ در آستانه انقراض قرار داشت و تعداد آن‌ها تنها به بیست تا سی ببر کاهش یافته بود. اما این حیوان به طرزی باورنکردنی از انقراض قریب‌الوقوع رهایی جست و جمعیت آن تا سال ۲۰۱۰ به حدود ۳۶۰ ببر رسید. ببر سیبری با توجه به همین افزایش جمعیت از سال ۲۰۱۰ از بالاترین ردهٔ حفاظتی یعنی «به شدت در معرض خطر» خارج شده و در یک رده پایین‌تر یعنی «در خطر انقراض» قرار گرفته است. ببرهای سیبری تنوع ژنتیکی بسیار پائینی دارند که این به دلیل کاهش شدید جمعیت این حیوان در دهه ۱۹۴۰ و تعداد اندک توله ببرهایی است که به بلوغ می‌رسند. ضمن اینکه بی

In [8]:
corpus = [(path.stem, path.open().read()) for path in Path('IR_dataset').iterdir()]
corpus = pd.DataFrame(corpus, columns=['docId','text'])

## Qrels

In [27]:
!wget https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml

--2021-12-28 08:01:03--  https://raw.githubusercontent.com/language-ml/2-LM-embedding-projects/main/problem3/evaluation_IR.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50854 (50K) [text/plain]
Saving to: ‘evaluation_IR.yml.1’

evaluation_IR.yml.1 100%[===================>]  49.66K  --.-KB/s    in 0.01s   

2021-12-28 08:01:03 (4.58 MB/s) - ‘evaluation_IR.yml.1’ saved [50854/50854]



In [29]:
query_data = yaml.safe_load(open('evaluation_IR.yml'))

In [34]:
query = list(query_data.keys())
relevant = [query_data[k]['relevant'][0] for k in query_data]
qrels = [{'query_id':q, 'doc_id':d,
          'relevance':3} for q in query for d in query_data[q]['similar_high']]
qrels.extend([{'query_id':q, 'doc_id':d,
          'relevance':2} for q in query for d in query_data[q]['similar_med']])
qrels = [{'query_id':q, 'doc_id':d,
          'relevance':1} for q in query for d in query_data[q]['similar_low']]
qrels.extend([{'query_id':q, 'doc_id':query_data[q]['relevant'], 'relevance':4} for q in query])
qrels = pd.DataFrame(qrels)

In [37]:
qrels.head()

,query_id,doc_id,relevance
0,آدولف هیتلر شکست و مرگ,355,1
1,آدولف هیتلر شکست و مرگ,356,1
2,آدولف هیتلر شکست و مرگ,357,1
3,آدولف هیتلر شکست و مرگ,358,1
4,آدولف هیتلر شکست و مرگ,359,1


In [ ]:
res = {
    'q0':{
        'doc0': 1.,
        'doc1': 2.,
        'doc3': 5.
    },
    'q1':{
        'doc0': 4.,
        'doc1': 6.,
        'doc2': 2.
    }
}
qrels = {
    'q0':{
        'doc0': 1,
        'doc1': 1,
        'doc3': 1
    },
    'q1':{
        'doc0': 0,
        'doc1': 2,
        'doc2': 2
    }
}
IRm.calc_aggregate([IRm.AP(rel=0)],qrels,res)

## Normaliztion

# Embedding the documents

## Method 1 : Tfidf

In [40]:
vectorizer = TfidfVectorizer(max_features=500,ngram_range=(1,2))
vectorizer.fit(corpus.text)

## Method 2 : Neural Network (RoBERTa)

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class RoBERTaVectorizer(TransformerMixin):
  def __init__(self):
    super().__init__()
    self.tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")
    self.model = AutoModelForMaskedLM.from_pretrained("HooshvareLab/roberta-fa-zwnj-base")\
                                     .to(DEVICE)
  def fit(self,x):
    pass
  def transform(self,x: List[str]):
    encoded_batch = self.tokenizer.batch_encode_plus(x, return_tensors='pt', padding=True)
    encoded_batch = {k: v.to(DEVICE) for k, v in encoded_batch.items()}
    with torch.no_grad():
      output = self.model(**encoded_batch)
    return output

# Document Retrieval

In [ ]:
class KNN_based_IR(BaseEstimator):
  def __init__(self,n_neighbors=1+10+10+10) -> None:
    super().__init__()
    self.nn = NearestNeighbors(n_neighbors=n_neighbors)
  def set_params(self,**kwargs):
    self.nn.set_params(**kwargs)
  def fit(self, X, y):
    self.nn.fit(X)
  def predict(self, X):
    scores, docIds = self.nn.kneighbors(X)
    return {k:(scores[v],docIds[v]) for k,v in zip(
        ['relevant','high_similar','med_similar','low_similar'],
        [slice(0),slice(1,11),slice(11,21),slice(21,31)])}

# IR Evaluation

## MRR (Mean Reciprocal Rank)

In [45]:
MRR = IRm.measures.MRR(rel=4)
def mrr(qrels, ret):
  return MRR.calc_aggregate(qrels, ret)
mrr_scorer = make_scorer(mrr)

## MAP (Mean Average Precision)

In [ ]:
map_scorer = make_scorer(average_precision_score,average='samples')

# Pipeline Definition

In [ ]:
pipeline = Pipeline([('embedding','passthrough'),
                     ('retrieval','passthrough')])